<a href="https://colab.research.google.com/github/HerbertMariano/time-series-forecasting/blob/main/mancha_solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')

In [3]:
df.sort_values('Month',inplace=True)
df.set_index('Month',inplace=True)

In [4]:
features = ['Sunspots']
target = []

In [5]:
for i in range(1,12):
    df[f'Sunspots-{i}'] = df['Sunspots'].shift(i)
    features.append(f'Sunspots-{i}')

for i in range(1,3):
    df[f'target-{i}'] = df['Sunspots-11'].shift(-i)
    target.append(f'target-{i}')

In [6]:
df.dropna(inplace=True)

In [9]:
model = XGBRegressor()

for i in range(4,1,-1):
    X = df[features][:-i*12]
    y = df[target][:-i*12]
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    search = BayesSearchCV(model, {'n_estimators': (100, 1000),
                                   'max_depth': (1, 10),
                                   'learning_rate': (0.001, 1.0, 'log-uniform'),
                                   'subsample': (0.01, 1.0, 'uniform'),
                                   'colsample_bytree': (0.01, 1.0, 'uniform'),
                                   'min_child_weight': (0, 10)}, n_iter=32, cv=2, n_jobs=-1)
    search.fit(X, y)
    print(f'Best score for target-{i}: {search.best_score_}')
    print(f'Best parameters for target-{i}: {search.best_params_}')

    

Best score for target-4: 0.9996989241615453
Best parameters for target-4: OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.09614373919479663), ('max_depth', 10), ('min_child_weight', 0), ('n_estimators', 100), ('subsample', 1.0)])
Best score for target-3: 0.9982270030697509
Best parameters for target-3: OrderedDict([('colsample_bytree', 0.83825023230939), ('learning_rate', 0.17648679548363413), ('max_depth', 6), ('min_child_weight', 0), ('n_estimators', 310), ('subsample', 0.8324232518895744)])
Best score for target-2: 0.9975443322727664
Best parameters for target-2: OrderedDict([('colsample_bytree', 0.8959065928286939), ('learning_rate', 0.4899108626838077), ('max_depth', 9), ('min_child_weight', 1), ('n_estimators', 953), ('subsample', 0.932738250522197)])
